In [2]:
gpu_info = !nvidia-smi -i 0
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from torchvision import transforms
import numpy as np

Thu Mar 30 20:47:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
train_transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(), 
    # transforms.RandomResizedCrop(size=28, scale=(0.2, 1.)),
    transforms.ToTensor(), 
    transforms.Normalize((0.1307, ), (0.3081,))])


test_transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(), 
    # transforms.RandomResizedCrop(size=28, scale=(0.2, 1.)),
    transforms.ToTensor(), 
    transforms.Normalize((0.1307, ), (0.3081))])


In [19]:
# data prepare
dataset = MNIST(root='./DATA', train=True, transform=train_transform, download=True)
train_data = torch.utils.data.Subset(dataset, range(100))
train_loader = DataLoader(train_data, batch_size=50, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)

# memory_data = MNIST(root='./DATA', train=True, transform=test_transform, download=True)
# memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=10, pin_memory=True)

test_data = MNIST(root='./DATA', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=50, shuffle=False, num_workers=10, pin_memory=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/train-images-idx3-ubyte.gz to ./DATA/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/train-labels-idx1-ubyte.gz to ./DATA/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/t10k-images-idx3-ubyte.gz to ./DATA/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./DATA/MNIST/raw



/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
class Encoder(torch.nn.Module):
    "Encoder network"
    def __init__(self):
        super(Encoder, self).__init__()
        # L1 (?, 28, 28, 1) -> (?, 28, 28, 32) -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.2)
            )
        # L2 (?, 14, 14, 32) -> (?, 14, 14, 64) -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.2)
            )
        # L3 (?, 7, 7, 64) -> (?, 7, 7, 128) -> (?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            torch.nn.Dropout(p=0.2)
            )
        self._to_linear = 4 * 4 * 128
        self.fc = torch.nn.Linear(in_features=4 * 4 * 128, out_features=128)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1) # Flatten them for FC
        x = self.fc(x)
        return x

In [5]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.encoder_q = Encoder()
        self.fl = torch.nn.Linear(in_features=128, out_features=10)
    def forward(self, x):
      x = self.encoder_q(x)
      x = self.fl(x)

      return x

In [6]:
down_stream = Classifier()

In [7]:
for name, param in down_stream.encoder_q.named_parameters():
  if name not in ["fc.weight", "fc.bias"]:
    param.requires_grad = False

In [8]:
for name, param in down_stream.named_parameters():
  if param.requires_grad:
    print(name)

encoder_q.fc.weight
encoder_q.fc.bias
fl.weight
fl.bias


In [9]:
down_stream.encoder_q.load_state_dict(torch.load("./moco/moco_final_q.pt"))
down_stream.encoder_q.eval()

Encoder(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
 

In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" #0 indicates the GPU you gonna use 
print(f"Using {device} device")

Using cuda:0 device


In [11]:
criterion =  nn.CrossEntropyLoss().cuda()
# Optimizing method
optimizer = torch.optim.Adam(down_stream.parameters(), lr=0.001)
down_stream.to(device)

Classifier(
  (encoder_q): Encoder(
    (layer1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.2, inplace=False)
    )
    (layer2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.2, inplace=False)
    )
    (layer3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=1, dila

In [12]:
!pip install optuna --quiet
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.0 MB/s eta 0:00:00


In [13]:
def objective(trial):
  


    # Define the search space for hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 3e-1, log=True)
    # wd = trial.suggest_float('wd', 5e-4, 0.1, log=True)
    # m = trial.suggest_float('mom_encod', 0.0, 1.0)
    # t = trial.suggest_float('temperature', 0.1, 1.0)
    batch_size = trial.suggest_categorical('batch_size', [10, 32, 50])
    #K = trial.suggest_categorical('num_negatives', [6656])#4096, 6656])
    # sym = trial.suggest_categorical('symmetric', [True, False])
    # momentum = trial.suggest_float('momentum', 0.1, 1.0)
    # Load the dataset
    # data prepare
    dataset = MNIST(root='./DATA', train=True, transform=train_transform, download=True)
    train_data = torch.utils.data.Subset(dataset, range(100))
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

    # memory_data = MNIST(root='./DATA', train=True, transform=test_transform, download=True)
    # memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=10, pin_memory=True)

    test_data = MNIST(root='./DATA', train=False, transform=test_transform, download=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=10, pin_memory=True)

   #defining the moco model
    

    # Defining the optimizer
    optimizer = torch.optim.Adam(down_stream.parameters(), lr=lr)


    # Training the MoCo model
    epoch_start = 1
    epochs = 75

    for epoch in tqdm(range(epochs)) :
  # Training
      down_stream.train()
      running_loss = 0.0
      for idx, batch in enumerate(train_loader):
        inputs, labels = batch
        inputs = inputs.cuda()
        labels = labels.cuda()

        #zero the parameter grad|ients
        optimizer.zero_grad()

        #forward + backward + optimize

        out = down_stream(x = inputs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        #print statistics
        running_loss += loss.item()
        
      correct = 0
      total = 0
      with torch.no_grad():
        for images, labels in test_loader:
          images = images.cuda()
          labels = labels.cuda()
          outputs = down_stream(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
   
    return 100*correct / total 
  
   

  
# Creating an Optuna study and optimizing the hyperparameters
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=500)

# Printing the best hyperparameters and the corresponding train loss
print('Best hyperparameters: ', study.best_params)
print('Best validation loss: {:.4f}'.format(study.best_value))

[I 2023-03-30 20:49:01,250] A new study created in memory with name: no-name-d686170d-9f1f-4472-8a29-15b8b4aca6fc


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/train-images-idx3-ubyte.gz to ./DATA/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/train-labels-idx1-ubyte.gz to ./DATA/MNIST/raw



...

Extracting ./DATA/MNIST/raw/t10k-images-idx3-ubyte.gz to ./DATA/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./DATA/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./DATA/MNIST/raw



100%|██████████| 75/75 [01:56<00:00,  1.56s/it]
[I 2023-03-30 20:51:00,105] Trial 0 finished with value: 70.56 and parameters: {'lr': 0.22288961918928699, 'batch_size': 32}. Best is trial 0 with value: 70.56.
100%|██████████| 75/75 [03:34<00:00,  2.86s/it]
[I 2023-03-30 20:54:34,345] Trial 1 finished with value: 70.16 and parameters: {'lr': 1.121702566953417e-05, 'batch_size': 10}. Best is trial 0 with value: 70.56.
100%|██████████| 75/75 [01:34<00:00,  1.27s/it]
[I 2023-03-30 20:56:09,389] Trial 2 finished with value: 77.38 and parameters: {'lr': 0.05984430384741256, 'batch_size': 50}. Best is trial 2 with value: 77.38.
100%|██████████| 75/75 [03:33<00:00,  2.84s/it]
[I 2023-03-30 20:59:42,566] Trial 3 finished with value: 76.54 and parameters: {'lr': 2.529354732749093e-05, 'batch_size': 10}. Best is trial 2 with value: 77.38.
100%|██████████| 75/75 [03:34<00:00,  2.86s/it]
[I 2023-03-30 21:03:17,178] Trial 4 finished with value: 76.91 and parameters: {'lr': 0.0017450910436455933, 'ba

KeyboardInterrupt: ignored

In [ ]:
encoder_tsne = Encoder()
encoder_tsne.load_state_dict(torch.load(""))